In [1]:
# import the project1-prepareData notebook:
!pip install ipynb
from ipynb.fs.full.project1_prepareData import *


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


SyntaxError: invalid syntax (<ipython-input-1-2f7628696289>, line 3)

In [ ]:
# Filter bad rows to include only relevant data that would make the AUC higher.
# We will also use only USA records because it contains 30162 rows and it's more levant for an average income of $50k
# We are trying different columns to find best combination to get a more accurate model.

categorical_columns_selected = [
    'workclass', # Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
    'education', # Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
    'marital-status', # Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
    'occupation', # Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
    'relationship', # Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
    'race', # White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
    'sex', # Female, Male.
#     'native-country', # United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands
]
numerical_columns_selected = [
    'age', # continuous.
#     'fnlwgt', # continuous. final weight based on Gov CPS data
    'education-num', # continuous.
#     'capital-gain', # continuous.
#     'capital-loss', # continuous.
    'hours-per-week', # continuous.
]

y_column = 'income'


#Trying to reduce the education types lower than highscool but AUC didn't increase

# 'Assoc-acdm', 'Assoc-voc', 'Preschool'
# education_compact = ['11th', '9th', '7th-8th', '12th', '1st-4th', '10th', '5th-6th']
# df_clean.education.replace(dict.fromkeys(education_compact, 'pre-high-school'), inplace=True)
# df_clean["low-education"]  = df_clean.education.isin(education1)

#Filtering only USA data
df_clean_filtered = df_clean.copy()
df_clean_filtered = df_clean_filtered[df_clean_filtered['native-country'] == 'United-States']
# df_clean_filtered = df_clean_filtered[df_clean_filtered['education'] != 'Preschool']
df_clean_filtered = df_clean_filtered[categorical_columns_selected+numerical_columns_selected+['low_income']]
display("Selected data:", df_clean_filtered)


In [ ]:
# Split data

from sklearn.model_selection import train_test_split

df_full_train, df_test = train_test_split(df_clean_filtered, test_size=0.2, random_state=11)
df_train,  df_val = train_test_split(df_full_train, test_size=0.25, random_state=11)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = (df_train.low_income == True).astype('int').values
y_val = (df_val.low_income == True).astype('int').values
y_test = (df_test.low_income == True).astype('int').values

del df_train["low_income"]
del df_val["low_income"]
del df_test["low_income"]

# with pd.option_context('display.max_rows', 2, 'display.max_columns', None): 
#     display(df_test)   
    
print("df_train length: ", len(df_train))
print("df_val length: ", len(df_val))
print("df_test length: ", len(df_test))

print("y_train length: ", len(y_train))
print("y_val length: ", len(y_val))
print("y_test length: ", len(y_test))


In [2]:
#Hot encoding and Logistic regretion
from sklearn import linear_model
from sklearn.feature_extraction import DictVectorizer

def train(dataFrame, y):
    # Hot Encoding
    dicts = dataFrame.to_dict(orient="records")
    dv = DictVectorizer(sparse=False)
    X = dv.fit_transform(dicts)

    # train
    model = linear_model.LogisticRegression()
    model.fit(X, y)
    return dv, model


dv, model = train(df_val, y_val)
weights = model.coef_[0].round(3) # weights
w0 = model.intercept_ # bias, w0

weights_with_featureNames = dict(zip(dv.get_feature_names(), weights))
print("w0 =", w0)
# display(weights_with_featureNames)
display(pd.DataFrame([weights], index=["weight"], columns=dv.get_feature_names()))



NameError: name 'df_val' is not defined

In [ ]:
# Predict

def predict(dataFrame, dv, model):
    dicts = dataFrame.to_dict(orient="records")
    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:,1]
    return y_pred


proba = predict(df_val, dv, model)
y_pred_val = proba  #take column 1

low_salary_pred = (y_pred_val >= 0.5)
# customers_with_predicted_low_income = df_val[low_salary_pred]

In [ ]:
# Check accuracy

#check average accuracy on y_val
df_pred = pd.DataFrame()
df_pred["probability"] = y_pred_val
df_pred["prediction"] = low_salary_pred.astype(int)
df_pred["actual"] = y_val
df_pred["prediction_correct"] = df_pred.prediction == df_pred.actual
display(df_pred)
print("Accuracy % on y_val:",df_pred.prediction_correct.mean())

In [ ]:
# AUC and ROC curve

from sklearn import metrics

fpr, tpr, thresholds = metrics.roc_curve(y_val, y_pred_val)
auc = metrics.auc(fpr, tpr)

# print("thresholds:", thresholds)
print("AUC for LogisticRegresion:", auc)



import matplotlib.pyplot as plt
from sklearn import datasets, model_selection, svm

dicts = df_val.to_dict(orient="records")
X_val = dv.transform(dicts)
print("ROC curve:")
metrics.plot_roc_curve(model, X_val, y_val) 
plt.show()
